In [2]:
from __future__ import print_function, division

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['figure.figsize'] = np.array((5,4))*1.5
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(context="poster", style="ticks")
import os
import glob



from units import M_solar, m_proton, pc, yr, Myr, km, s, gamma

from injection_helpers import get_SNe

from visualize_helpers import \
    total_radial_momentum_of_snapshot, \
    map_to_all_snapshots, \
    get_snapshot_times, \
    load_snapshots, \
    load_ds_from_ts, \
    get_snapshot_filenames, \
    snapshot_filename_to_number
    
from sql_helpers import add_simulation, \
    open_as_DataFrame, get_db_dirname_tmp


`grackle_helpers.wrapped_initializer` is setting up grackle assuming code units:
    mass   : 1.0 M_solar
    length : 1.0 pc
    time   : 1.0 Myr

grackle cooling file:  b'/pfs/home/egentry/local/grackle/input/CloudyData_UVB=HM2012.h5'


In [3]:
def ratchet_filter(data):
    derivatives = np.append(0, np.diff(data))
    mask = derivatives < 0
    derivatives[mask] = 0
    
    return derivatives.cumsum()

In [4]:
db_dir_tmp = get_db_dirname_tmp()
if not os.path.exists(db_dir_tmp):
    print("making temporary database directory: ", db_dir_tmp)
    os.mkdir(db_dir_tmp)

making temporary database directory:  /dev/shm/egentry


In [5]:
def copy_and_get_DataFrames(run_name, verbose=True):
    add_simulation(run_name, verbose=verbose)
    
    df = open_as_DataFrame(run_name, copy_first=True)
    
    return df

In [6]:
## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        
from clustered_SNe.analysis.parse import Overview, RunSummary

In [7]:
data_dir_1D = "1D_data/"

run_summary = RunSummary(data_dir_1D, "F5509BF1-3F9E-4008-B795-0482ECED199B")

In [8]:
run_names = [
    "cluster_cooling_400",
    "cluster_cooling_200",
    "cluster_cooling_100",
    "cluster_cooling_mhd_large_200",
]

In [9]:
run_name_to_run_id = {
    "1D" : "1D_06_HD",
    "cluster_cooling_400" : "3D_10_HD",
    "cluster_cooling_200" : "3D_20_HD",
    "cluster_cooling_100" : "3D_40_HD",
    "cluster_cooling_mhd_large_200" : "3D_20_MHD",
}

def texify_run_id(run_id):
    return "\\texttt{{{}\_{}\_{}}}".format(*run_id.split("_"))

In [10]:
sorted_run_ids = [
    run_name_to_run_id[run_name]
    for run_name in (["1D"] + run_names)
]
sorted_run_ids

['1D_06_HD', '3D_10_HD', '3D_20_HD', '3D_40_HD', '3D_20_MHD']

In [11]:
SNe_dir = "../runs/{}/inputs/".format(run_names[0])
SNe = get_SNe(SNe_dir)
SN_times = np.array([SN.time for SN in SNe])

In [12]:
dfs = [
    copy_and_get_DataFrames(run_name, verbose=False)
    for run_name in run_names
]

In [17]:
results_filename = "results_table.csv"

# if os.path.exists(results_filename):
if False:
    df_results = pd.read_csv(results_filename)
else:
    results = {}
    
    # first do the 1D simulation, which has to be processed separately
    # but use the same "R_eff" definition as the 3D runs, rather than the
    # "R_shock" definition I used in our previous paper.
    # This'll let us be internally consistent in this paper, and shouldn't
    # affect the results much anyway
    
    results_tmp = {}
    i_max = np.argmax(run_summary.momentum)
    results_tmp["max_snapshot"] = i_max
    results_tmp["time"] = run_summary.times[i_max] / Myr

    df_tmp = run_summary.df.loc[99]
    df_tmp = df_tmp[np.abs(df_tmp.Velocity) > 100]

    results_tmp["M_affected"] = df_tmp.Mass.sum()
    results_tmp["R_eff"] = (df_tmp.dV.sum() / (4*np.pi/3))**(1/3) / pc
    results_tmp["momentum_per_SN"] = run_summary.momentum[i_max] / (100 * M_solar * km/s * 11)
    results_tmp["momentum_per_SN_ratchet"] = ratchet_filter(run_summary.momentum).max() / (100 * M_solar * km/s * 11)
    results_tmp["dE_kin"] = run_summary.E_kin[i_max] - run_summary.E_kin[0]
    results_tmp["dE_int"] = run_summary.E_int[i_max] - run_summary.E_int[0] + 1e51
    
    results[run_name_to_run_id["1D"]] = results_tmp
    
    # now go on to the 3D simulations

    for run_name, df in zip(run_names, dfs):
        run_id = run_name_to_run_id[run_name]
        mask = df.time >  SN_times.max()

        i_max = np.argmax(df[mask].momentum)

        row_0 = df.loc[0]
        row = df.loc[i_max]

        dE_int = row.e_int - row_0.e_int
        dE_kin = row.e_kin - row_0.e_kin

        outputs_dir = "../runs/{}/outputs/".format(run_name)
        print(outputs_dir)

        snapshot_filenames = get_snapshot_filenames(outputs_dir)

        snapshot_number_to_index_map = {snapshot_filename_to_number(filename) : i
                                        for i,filename in enumerate(snapshot_filenames) }

        ts = load_snapshots(outputs_dir)
        ds = load_ds_from_ts(ts, snapshot_number_to_index_map[i_max])
        dd = ds.all_data()
        V_affected = (dd["affected", "Masses"] / dd["affected", "Density"]).sum()
        M_affected = float((dd["affected", "Masses"]).sum().to("Msun").value)


        R_eff = (V_affected / (4*np.pi/3))**(1/3)
        R_eff = float(R_eff.to("pc").value)

        del dd

        results[run_id] = {
            "max_snapshot" : i_max,
            "time" : row.time,
            "R_eff" : R_eff,
            "M_affected" : M_affected,
            "momentum_per_SN" : row.momentum / (100 * M_solar * km/s * SN_times.size),
            "momentum_per_SN_ratchet" : ratchet_filter(df.momentum).max() / (100 * M_solar * km/s * SN_times.size),
            "dE_kin" : dE_kin, 
            "dE_int" : dE_int, 
        }

        print("{:.1f} {:.1f} {:.2e} {:.1f} {:.1f} {:.2e} {:.2e}".format(
            results[run_id]["time"],
            results[run_id]["R_eff"],
            results[run_id]["M_affected"],
            results[run_id]["momentum_per_SN"],
            results[run_id]["momentum_per_SN_ratchet"],
            results[run_id]["dE_kin"],
            results[run_id]["dE_int"],
        ))
        

    df_results = pd.DataFrame(data={
        "run_id" : sorted_run_ids,
        "max_snapshot" : [results[run_id]["max_snapshot"] for run_id in sorted_run_ids],
        "time" : [results[run_id]["time"] for run_id in sorted_run_ids],
        "R_eff" : [results[run_id]["R_eff"] for run_id in sorted_run_ids],
        "M_affected" : [results[run_id]["M_affected"] for run_id in sorted_run_ids],
        "momentum_per_SN" : [results[run_id]["momentum_per_SN"] for run_id in sorted_run_ids],
        "momentum_per_SN_ratchet" : [results[run_id]["momentum_per_SN_ratchet"] for run_id in sorted_run_ids],
        "dE_kin" : [results[run_id]["dE_kin"] for run_id in sorted_run_ids],
        "dE_int" : [results[run_id]["dE_int"] for run_id in sorted_run_ids],
    })

    df_results = df_results[["run_id", "max_snapshot", "time", "R_eff", "M_affected", "momentum_per_SN", "momentum_per_SN_ratchet", "dE_kin", "dE_int"]]
    df_results.to_csv(results_filename, index=False)
    
df_results.head()

../runs/cluster_cooling_400/outputs/


yt : [INFO     ] 2017-11-19 15:26:08,218 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2017-11-19 15:26:08,249 Parameters: current_time              = 30.6618895507
yt : [INFO     ] 2017-11-19 15:26:08,251 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2017-11-19 15:26:08,253 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-11-19 15:26:08,255 Parameters: domain_right_edge         = [ 600.  600.  600.]
yt : [INFO     ] 2017-11-19 15:26:08,257 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2017-11-19 15:26:08,265 Allocating for 2.160e+08 particles (index particle type 'all')
yt : [INFO     ] 2017-11-19 15:26:44,053 Identified 1.726e+07 octs


30.7 218.1 1.67e+06 2425.1 2474.3 8.73e+49 8.38e+48
../runs/cluster_cooling_200/outputs/


yt : [INFO     ] 2017-11-19 15:29:21,868 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2017-11-19 15:29:21,886 Parameters: current_time              = 30.6618895216
yt : [INFO     ] 2017-11-19 15:29:21,887 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2017-11-19 15:29:21,889 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-11-19 15:29:21,890 Parameters: domain_right_edge         = [ 600.  600.  600.]
yt : [INFO     ] 2017-11-19 15:29:21,891 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2017-11-19 15:29:21,896 Allocating for 2.700e+07 particles (index particle type 'all')
yt : [INFO     ] 2017-11-19 15:29:32,712 Identified 2.167e+06 octs


30.7 200.2 1.48e+06 2127.5 2182.0 7.37e+49 8.47e+48
../runs/cluster_cooling_100/outputs/


yt : [INFO     ] 2017-11-19 15:30:07,498 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2017-11-19 15:30:07,519 Parameters: current_time              = 31.6993462555
yt : [INFO     ] 2017-11-19 15:30:07,521 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2017-11-19 15:30:07,523 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-11-19 15:30:07,524 Parameters: domain_right_edge         = [ 600.  600.  600.]
yt : [INFO     ] 2017-11-19 15:30:07,526 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2017-11-19 15:30:07,536 Allocating for 3.375e+06 particles (index particle type 'all')
yt : [INFO     ] 2017-11-19 15:30:08,826 Identified 2.697e+05 octs


31.7 209.2 1.79e+06 2006.7 2039.0 6.81e+49 6.98e+49
../runs/cluster_cooling_mhd_large_200/outputs/


yt : [INFO     ] 2017-11-19 15:30:14,428 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2017-11-19 15:30:14,468 Parameters: current_time              = 29.6244328246
yt : [INFO     ] 2017-11-19 15:30:14,470 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2017-11-19 15:30:14,473 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-11-19 15:30:14,476 Parameters: domain_right_edge         = [ 1200.  1200.  1200.]
yt : [INFO     ] 2017-11-19 15:30:14,479 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2017-11-19 15:30:14,488 Allocating for 2.160e+08 particles (index particle type 'all')
yt : [INFO     ] 2017-11-19 15:31:36,268 Identified 1.886e+07 octs


29.6 423.0 1.05e+07 1213.2 2418.1 1.26e+50 1.31e+50


,run_id,max_snapshot,time,R_eff,M_affected,momentum_per_SN,momentum_per_SN_ratchet,dE_kin,dE_int
0,1D_06_HD,99,94.775395,552.455190,2.321468e+07,33978.066772,33978.066772,6.496970e+50,2.625387e+50
1,3D_10_HD,95,30.661890,218.077572,1.672155e+06,2425.087581,2474.279148,8.732023e+49,8.383656e+48
2,3D_20_HD,95,30.661890,200.244034,1.483986e+06,2127.527079,2182.029716,7.370499e+49,8.470866e+48
3,3D_40_HD,97,31.699346,209.174645,1.788160e+06,2006.743048,2039.021903,6.812285e+49,6.983841e+49
4,3D_20_MHD,94,29.624433,422.989193,1.045209e+07,1213.207419,2418.063520,1.264744e+50,1.311465e+50


In [18]:
!cat $results_filename

run_id,max_snapshot,time,R_eff,M_affected,momentum_per_SN,momentum_per_SN_ratchet,dE_kin,dE_int
1D_06_HD,99,94.77539483357417,552.4551897601157,23214681.41063985,33978.06677183907,33978.06677183907,6.496970059706205e+50,2.6253872396302327e+50
3D_10_HD,95,30.661889550677877,218.0775724536534,1672155.2484314037,2425.087580872953,2474.2791475932663,8.732023081003739e+49,8.383655848412868e+48
3D_20_HD,95,30.66188952164082,200.24403436363107,1483985.5399279275,2127.5270786442948,2182.0297155442613,7.370498517587295e+49,8.470865615846771e+48
3D_40_HD,97,31.699346255460032,209.17464535867362,1788160.299437912,2006.7430481573908,2039.0219028012893,6.812284513733251e+49,6.983841108067664e+49
3D_20_MHD,94,29.624432824639175,422.98919294606577,10452086.801225144,1213.2074187756673,2418.0635203055467,1.2647439226449764e+50,1.3114647177832476e+50


# Print to LaTeX

In [19]:
for _, row in df_results.iterrows():
    line = "{:s} & {:d} & {:.1f} & {:.0f} & {:.1f} & {:.0f} & {:.0f} & {:.1f} & {:.1f} \\\\"
    line = line.format(
        texify_run_id(row.run_id),
        11,
        row.time,
        row.R_eff,
        row.M_affected / 1e6,
        row.momentum_per_SN,
        row.momentum_per_SN_ratchet,
        row.dE_kin / 1e49, 
        row.dE_int / 1e49,
    )
    
    print(line)


\texttt{1D\_06\_HD} & 11 & 94.8 & 552 & 23.2 & 33978 & 33978 & 65.0 & 26.3 \\
\texttt{3D\_10\_HD} & 11 & 30.7 & 218 & 1.7 & 2425 & 2474 & 8.7 & 0.8 \\
\texttt{3D\_20\_HD} & 11 & 30.7 & 200 & 1.5 & 2128 & 2182 & 7.4 & 0.8 \\
\texttt{3D\_40\_HD} & 11 & 31.7 & 209 & 1.8 & 2007 & 2039 & 6.8 & 7.0 \\
\texttt{3D\_20\_MHD} & 11 & 29.6 & 423 & 10.5 & 1213 & 2418 & 12.6 & 13.1 \\
